In [1]:
import secrets                                      #Genera números aleatorios seguros para trabajar con secretos criptográficos  (https://docs.python.org/es/3.8/library/secrets.html)
import string                                       #Operaciones comunes de cadena de caracteres                                  (https://docs.python.org/es/3.9/library/string.html)
import pyperclip                                    #Permite copiar y pegar texto al portapapeles de forma sencilla               (https://pyperclip.readthedocs.io/en/latest/)
import ipywidgets as widgets                        #Widgets interactivos para Jupyter Notebook                                   (https://ipywidgets.readthedocs.io/en/stable/)
from IPython.display import display                 #Para mostrar la interfaz de usuario en Jupyter Notebook                      (https://ipython.readthedocs.io/en/8.26.0/api/generated/IPython.display.html)

# ---------- Funciones comunes ----------
def evaluar_fortaleza(password):
    score = sum([
        any(c.islower() for c in password),
        any(c.isupper() for c in password),
        any(c.isdigit() for c in password),
        any(c in string.punctuation for c in password),
        len(password) >= 12
    ])
    return "Fuerte" if score >= 4 else "Media" if score == 3 else "Débil"

def mostrar_resultado(pw):
    fortaleza = evaluar_fortaleza(pw)
    colores = {"Fuerte": "green", "Media": "orange", "Débil": "red"}
    salida = widgets.VBox([
        widgets.Text(value=pw, disabled=True, description="Contraseña:"),
        widgets.HTML(f"<b style='color:{colores[fortaleza]}'>Fortaleza: {fortaleza}</b>"),
        widgets.Button(description="📋 Copiar al portapapeles", 
                       button_style="info", 
                       layout=widgets.Layout(width="250px"))
    ])
    
    def copiar_clipboard(_):
        try: pyperclip.copy(pw)
        except: pass
    
    salida.children[2].on_click(copiar_clipboard)
    return salida

def generar_y_mostrar(chars, longitud):
    pw = ''.join(secrets.choice(chars) for _ in range(longitud))
    mostrar_en_submenu(mostrar_resultado(pw))

# ---------- Salidas ----------
salida_principal = widgets.Output()
salida_submenu = widgets.VBox()  # ✅ CAMBIO CLAVE: antes era widgets.Output()

def mostrar_en_submenu(*widgets_to_display):
    salida_submenu.children = [widgets.VBox(list(widgets_to_display))]

# ---------- Generadores ----------
def generar_manual(_):
    encabezado = widgets.HTML("<h4>📝 Yo genero mi propia contraseña</h4>")
    txt_long = widgets.BoundedIntText(value=8, min=4, max=100, description="Longitud:")
    txt_pass = widgets.Text(description="Contraseña:")
    btn_val = widgets.Button(description="Validar", button_style="primary")

    def validar(_):
        pw = txt_pass.value
        if len(pw) != txt_long.value:
            mostrar_en_submenu(widgets.HTML("<b style='color:red'>❌ La longitud no coincide.</b>"))
        else:
            mostrar_en_submenu(mostrar_resultado(pw))

    btn_val.on_click(validar)
    mostrar_en_submenu(encabezado, txt_long, txt_pass, btn_val)

def generar_facil_decir(_):
    encabezado = widgets.HTML("<h4>🔡 Contraseña fácil de decir</h4>")
    long_slider = widgets.BoundedIntText(value=8, min=4, max=50, description="Longitud:")
    btn_gen = widgets.Button(description="Generar", button_style="success")

    btn_gen.on_click(lambda _: generar_y_mostrar(string.ascii_letters, long_slider.value))
    mostrar_en_submenu(encabezado, long_slider, btn_gen)

def generar_facil_leer(_):
    encabezado = widgets.HTML("<h4>🔤 Contraseña fácil de leer</h4>")
    long_slider = widgets.BoundedIntText(value=8, min=8, max=50, description="Longitud:")
    chk_num = widgets.Checkbox(description="Incluir números")
    chk_sim = widgets.Checkbox(description="Incluir símbolos")
    btn_gen = widgets.Button(description="Generar", button_style="success")

    def generar(_):
        chars = 'abcdefghjkmnpqrstuvwxyzABCDEFGHJKLMNPQRSTUVWXYZ'
        if chk_num.value: chars += '23456789'
        if chk_sim.value: chars += '!@#$%^&*'
        generar_y_mostrar(chars, long_slider.value)

    btn_gen.on_click(generar)
    mostrar_en_submenu(encabezado, long_slider, chk_num, chk_sim, btn_gen)

def generar_todos(_):
    encabezado = widgets.HTML("<h4>🌐 Contraseña con todos los caracteres</h4>")
    long_slider = widgets.BoundedIntText(value=12, min=8, max=50, description="Longitud:")
    chk_upper = widgets.Checkbox(value=True, description="Mayúsculas")
    chk_lower = widgets.Checkbox(value=True, description="Minúsculas")
    chk_num = widgets.Checkbox(value=True, description="Números")
    chk_sim = widgets.Checkbox(value=False, description="Símbolos")
    btn_gen = widgets.Button(description="Generar", button_style="success")

    def generar(_):
        grupos = []
        if chk_upper.value: grupos.append(string.ascii_uppercase)
        if chk_lower.value: grupos.append(string.ascii_lowercase)
        if chk_num.value: grupos.append(string.digits)
        if chk_sim.value: grupos.append(string.punctuation)

        if len(grupos) < 3:
            mostrar_en_submenu(widgets.HTML("<b style='color:red'>❗ Elige al menos 3 tipos de caracteres.</b>"))
            return

        pool = ''.join(grupos)
        generar_y_mostrar(pool, long_slider.value)

    btn_gen.on_click(generar)
    mostrar_en_submenu(encabezado, long_slider, chk_upper, chk_lower, chk_num, chk_sim, btn_gen)

# ---------- Navegación ----------
def mostrar_reglas(_):
    salida_principal.clear_output()
    with salida_principal:
        display(widgets.HTML("""
        <h4>📜 Reglas para generar contraseñas:</h4>
        <ul>
            <li>Longitud recomendada: mínimo 8 caracteres</li>
            <li>Incluye letras, números y símbolos para mayor seguridad</li>
            <li>Evita usar datos personales</li>
        </ul>
        """))

def salir(_):
    salida_principal.clear_output()
    contenedor_menu_principal.layout.display = 'none'
    contenedor_submenu.layout.display = 'none'
    with salida_principal:
        display(widgets.HTML("<h4>👋 ¡Gracias por usar el generador!</h4>"))

def volver_menu(_):
    contenedor_menu_principal.layout.display = 'flex'
    contenedor_submenu.layout.display = 'none'
    salida_submenu.children = []

def ir_a_submenu(_):
    contenedor_menu_principal.layout.display = 'none'
    contenedor_submenu.layout.display = 'flex'
    salida_submenu.children = []

# ---------- Botones y estructura ----------
def boton_con_icono(texto, emoji, callback):
    boton = widgets.Button(description=texto, layout=widgets.Layout(width='280px'), button_style="")
    boton.on_click(callback)
    return widgets.HBox([widgets.HTML(f"<span style='font-size:20px'>{emoji}</span>"), boton])

btn_generar = widgets.Button(description="🔐 Generar", button_style="primary")
btn_reglas = widgets.Button(description="📘 Ver reglas")
btn_salir = widgets.Button(description="🚪 Salir", button_style="danger")
btn_generar.on_click(ir_a_submenu)
btn_reglas.on_click(mostrar_reglas)
btn_salir.on_click(salir)

contenedor_menu_principal = widgets.VBox([
    widgets.HTML("<h3>🔐 Generador de Contraseñas Seguras</h3>"),
    widgets.HBox([btn_generar, btn_reglas, btn_salir]),
    salida_principal
])
contenedor_menu_principal.layout.display = 'flex'

contenedor_submenu = widgets.VBox([
    widgets.HTML("<h4>Selecciona el tipo de generación:</h4>"),
    boton_con_icono("Yo genero mi propia contraseña", "📝", generar_manual),
    boton_con_icono("Fácil de decir", "🔡", generar_facil_decir),
    boton_con_icono("Fácil de leer", "🔤", generar_facil_leer),
    boton_con_icono("Todos los caracteres", "🌐", generar_todos),
    boton_con_icono("Volver", "⬅️", volver_menu),
    widgets.HTML("<hr>"),
    salida_submenu
])
contenedor_submenu.layout.display = 'none'

# ---------- Mostrar interfaz ----------
display(contenedor_menu_principal)
display(contenedor_submenu)
